# 번역기 만들어보기  
## 목표  
  
1. 번역기 모델 학습에 필요한 텍스트 데이터 전처리가 잘 이루어졌다.  
구두점, 대소문자, 띄어쓰기 등 번역기 모델에 요구되는 전처리가 정상적으로 진행되었다.  
  
2. seq2seq 기반의 번역기 모델이 정상적으로 구동된다.  
seq2seq 모델 훈련결과를 그래프로 출력해보고, validation loss그래프가 우하향하는 경향성을 보이며 학습이 진행됨이 확인되었다.  
  
3. 테스트 결과 의미가 통하는 수준의 번역문이 생성되었다.  
테스트용 디코더 모델이 정상적으로 만들어졌으며,  
input(영어)와 output(프랑스어) 모두 한글로 번역해서 결과를 출력해보았고, 둘의 내용이 유사함을 확인하였다.  

In [92]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

import numpy as np
import os
import pandas as pd
import re


### 데이터 로드

In [93]:
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수 :',len(lines))
lines.head(5) 

전체 샘플의 수 : 197463


,eng,fra,cc
0,Go.,Va !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,Marche.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Go.,En route !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Go.,Bouge !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
4,Hi.,Salut !,CC-BY 2.0 (France) Attribution: tatoeba.org #5...


In [94]:
lines.sample(5) 

,eng,fra,cc
173000,Everybody knew she could speak English well.,Tout le monde savait qu'elle pouvait parler tr...,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
13860,How did it feel?,Qu'as-tu ressenti ?,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
182039,Half of the world’s population lives in cities.,La moitié de la population mondiale vit en ville.,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
26567,Tom despised Mary.,Tom méprisait Marie.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
90,Get up.,Lève-toi !,CC-BY 2.0 (France) Attribution: tatoeba.org #8...


CC는 필요없는 부분이니 제거, 데이터는 33000개를 사용합니다  
데이터 정제 후, 3000개는 테스트셋으로 사용합니다  

In [95]:
lines = lines[['eng', 'fra']][:33000] 
lines.head(5)

,eng,fra
0,Go.,Va !
1,Go.,Marche.
2,Go.,En route !
3,Go.,Bouge !
4,Hi.,Salut !


### 전처리  
프랑스어 악센트를 제거합니다  
? ! . 등 구두점은 문장에서 분리합니다  

In [96]:
import unicodedata
    
def to_ascii(s):
  # 프랑스어 악센트(accent) 삭제
  # 예시 : 'déjà diné' -> deja dine
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

해당 함수는 문자열에 비자간마크가 있을 때 삭제해주고, 문장을 유니코드형태로 정규화해줍니다 

https://www.fontspace.com/unicode/category/nonspacing-mark  
https://docs.python.org/ko/3/library/unicodedata.html  
https://wikidocs.net/86900  

In [97]:
def preprocess_sentence(sent):
    # 악센트 제거 함수 호출
    sent = to_ascii(sent.lower())
    # 단어와 구두점 사이에 공백 추가.
    # ex) "I am a student." => "I am a student ."
    sent = re.sub(r"([?.!,¿])", r" \1", sent)

    # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환.
    sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

    # 다수 개의 공백을 하나의 공백으로 치환
    sent = re.sub(r"\s+", " ", sent)
    return sent


함수가 잘 짜였는지 테스트해봅니다  

In [98]:
print(lines.eng[3579])
print(preprocess_sentence(lines.eng[3579]))
print(lines.fra[3579])
print(preprocess_sentence(lines.fra[3579]))

It was long.
it was long .
C'était long.
c etait long .


데이터셋 전체를 바꿔줍니다

In [99]:
lines.eng = lines.eng.apply(preprocess_sentence)
lines.fra = lines.fra.apply(preprocess_sentence)

lines.sample(5)

,eng,fra
26882,tom shifted gears .,tom a change de vitesse .
24676,i ll tell you why .,je vous dirai pourquoi .
3036,here we are .,nous y voila !
29028,he banged his knee .,il se cogna le genou .
23864,i felt very happy .,je me sentais tres heureuse .


프랑스 악센트가 제거되고, 모든 문자를 소문자화 하였으며, 온점과 단어 사이에 구분을 위한 공백을 넣었습니다

### 토큰화   
띄어쓰기 단위로 토큰화를 수행합니다  
디코더의 문장에 시작 토큰과 종료 토큰을 넣어줍니다  

In [100]:
#시작과 끝 토큰을 붙여줍니다
lines['fra_in'] = lines['fra'].apply(lambda x: "<sos> "+ x)
lines['fra_out'] = lines['fra'].apply(lambda x: x + " <eos>")

lines.sample(5)

,eng,fra,fra_in,fra_out
20556,that s very true .,c est tres vrai .,<sos> c est tres vrai .,c est tres vrai . <eos>
16309,tom is unafraid .,tom ne craint rien .,<sos> tom ne craint rien .,tom ne craint rien . <eos>
19024,i play the piano .,je joue du piano .,<sos> je joue du piano .,je joue du piano . <eos>
2030,i need air .,il me faut de l air .,<sos> il me faut de l air .,il me faut de l air . <eos>
32677,tom is still young .,tom est encore jeune .,<sos> tom est encore jeune .,tom est encore jeune . <eos>


In [101]:
#단어별로 분리
lines.eng = lines.eng.apply(lambda x: x.split(' '))
lines.fra = lines.fra.apply(lambda x: x.split(' '))
lines.fra_in = lines.fra_in.apply(lambda x: x.split(' '))
lines.fra_out = lines.fra_out.apply(lambda x: x.split(' '))

lines.sample(5)

,eng,fra,fra_in,fra_out
5660,"[my, eyes, itch, .]","[mes, yeux, me, demangent, .]","[<sos>, mes, yeux, me, demangent, .]","[mes, yeux, me, demangent, ., <eos>]"
16583,"[we, are, students, .]","[nous, sommes, etudiants, .]","[<sos>, nous, sommes, etudiants, .]","[nous, sommes, etudiants, ., <eos>]"
12198,"[tom, is, thirsty, .]","[tom, est, assoiffe, .]","[<sos>, tom, est, assoiffe, .]","[tom, est, assoiffe, ., <eos>]"
11461,"[many, fish, died, .]","[beaucoup, de, poissons, ont, peri, .]","[<sos>, beaucoup, de, poissons, ont, peri, .]","[beaucoup, de, poissons, ont, peri, ., <eos>]"
5340,"[i, m, reformed, .]","[je, me, suis, amendee, .]","[<sos>, je, me, suis, amendee, .]","[je, me, suis, amendee, ., <eos>]"


In [102]:
lines.fra_in.values

array([list(['<sos>', 'va', '!']), list(['<sos>', 'marche', '.']),
       list(['<sos>', 'en', 'route', '!']), ...,
       list(['<sos>', 'nous', 'avons', 'aussi', 'trouve', 'ceci', '.']),
       list(['<sos>', 'nous', 'sommes', 'des', 'gens', 'occupes', '.']),
       list(['<sos>', 'nous', 'regardons', 'la', 'television', '.'])],
      dtype=object)

전처리가 잘 되었음을 확인하였습니다  

## 케라스 tokenizer 적용하기   
단어 기준이 되기 때문에, option의 char level은 사용하지 않습니다

In [116]:
eng_tokenizer = Tokenizer(filters="") 
eng_tokenizer.fit_on_texts(lines.eng)
   # 단어를 숫자값 인덱스로 변환하여 저장

In [117]:
fra_tokenizer = Tokenizer(filters="")  
fra_tokenizer.fit_on_texts(lines.fra_in)
fra_tokenizer.fit_on_texts(lines.fra_out)
   # 단어를 숫자값 인덱스로 변환하여 저장

In [118]:
encoder_input = eng_tokenizer.texts_to_sequences(lines.eng) 
decoder_input = fra_tokenizer.texts_to_sequences(lines.fra_in)  
decoder_target = fra_tokenizer.texts_to_sequences(lines.fra_out)

encoder_input =  pad_sequences(encoder_input, padding="post")
decoder_input =  pad_sequences(decoder_input, padding="post")
decoder_target =  pad_sequences(decoder_target, padding="post")

데이터의 크기를 확인하겠습니다

In [119]:
print('인코더의 입력의 크기(shape) :',encoder_input.shape)
print('디코더의 입력의 크기(shape) :',decoder_input.shape)
print('디코더의 레이블의 크기(shape) :',decoder_target.shape)

인코더의 입력의 크기(shape) : (33000, 8)
디코더의 입력의 크기(shape) : (33000, 16)
디코더의 레이블의 크기(shape) : (33000, 16)


In [120]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)

영어 단어장의 크기 : 4673
프랑스어 단어장의 크기 : 8138


In [121]:
max_eng_seq_len = max([len(line) for line in encoder_input])
max_fra_seq_len = max([len(line) for line in decoder_input])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 16


In [122]:
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding='post')
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 8)
프랑스어 입력데이터의 크기(shape) : (33000, 16)
프랑스어 출력데이터의 크기(shape) : (33000, 16)


In [123]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 8, 4673)
프랑스어 입력데이터의 크기(shape) : (33000, 16, 8138)
프랑스어 출력데이터의 크기(shape) : (33000, 16, 8138)


In [127]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input_train))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input_train))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target_train))

영어 학습데이터의 크기(shape) : (30000, 8, 4673)
프랑스어 학습 입력데이터의 크기(shape) : (30000, 16, 8138)
프랑스어 학습 출력데이터의 크기(shape) : (30000, 16, 8138)


## 회고  
프로젝트 구현을 완료하지 못했습니다..  
NLP에 대해 모르는 것이 많아서 하나씩 공부하다 보니 배우는것이 많았던 시간이었지만  
아직도 많이 부족함을 느낍니다  
더욱 성장하도록 하겠습니다  

### Step 4. 임베딩 층(Embedding layer) 사용하기


In [130]:
embedding_dim = 64
hidden_units = 64



In [132]:
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(eng_vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(hidden_units, return_state=True)
#컨텍스트 벡터!!!!!!!!
encoder_outputs, enc_state_h, enc_state_c = encoder_lstm(enc_emb)

In [133]:
decoder_inputs = Input(shape=(None,))
dec_emb =  Embedding(fra_vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(hidden_units, return_state=True)
#컨텍스트 벡터!!!!!!!!
decoder_outputs, dec_state_h, dec_state_c = decoder_lstm(dec_emb)

Step 5. 모델 구현하기


첫번째 줄 : 우선 LSTM의 입력 텐서를 정의해줍니다. 입력 문장을 저장하게 될 변수 텐서입니다.

두번째 줄 : 256의 hidden_size를 가지는 LSTM 셀을 만들어줍니다. LSTM의 수용력(capacity)를 의미합니다. return_state = True를 해서 hidden state와 cell state를 리턴받을 수 있도록 합니다.

세번째 줄 : 입력 텐서를 사용하여 마지막 time step의 hidden state와 cell state를 결과로 받습니다.

네번째 줄 : 마지막 time step의 hidden state와 cell state를 encoder_states라는 하나의 변수에 저장해뒀습니다. 이를 디코더에 전달하면 되겠네요.

이제 디코더를 설계해볼까요? 디코더도 인코더랑 몇 가지 세부 사항을 제외하고 거의 똑같습니다.

In [ ]:
# 입력 텐서 생성.
encoder_inputs = Input(shape=(None, eng_vocab_size))
# hidden size가 256인 인코더의 LSTM 셀 생성
encoder_lstm = LSTM(units = 256, return_state = True)
# 디코더로 전달할 hidden state, cell state를 리턴. encoder_outputs은 여기서는 불필요.
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# hidden state와 cell state를 다음 time step으로 전달하기 위해서 별도 저장.
#이것이 컨텍스트 벡터!!!!!!!!
encoder_states = [state_h, state_c]

In [ ]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.summary()

In [ ]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128, epochs=50)

Step 6. 모델 평가하기


In [ ]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

In [ ]:
# 이전 time step의 hidden state를 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
# 이전 time step의 cell state를 저장하는 텐서
decoder_state_input_c = Input(shape=(256,))
# 이전 time step의 hidden state와 cell state를 하나의 변수에 저장
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# decoder_states_inputs를 현재 time step의 초기 상태로 사용.
# 구체적인 동작 자체는 def decode_sequence()에 구현.
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_states_inputs)
# 현재 time step의 hidden state와 cell state를 하나의 변수에 저장.
decoder_states = [state_h, state_c]

In [ ]:
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)
decoder_model.summary()

In [ ]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [ ]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, fra_vocab_size))
    target_seq[0, 0, fra2idx['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, fra_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [ ]:
import numpy as np
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스 (자유롭게 선택해 보세요)
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.eng[seq_index])
    print('정답 문장:', lines.fra[seq_index][1:len(lines.fra[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력